In [1]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import os
import warnings
from sklearn.model_selection import cross_validate

# 1. Importar Normalización Ganadora
from normalization_functions import pipeline_d_normalize
# 2. Importar Características Aumentadas 
from lexicon_functions import get_augmented_features

Recurso 'stopwords' de NLTK ya está descargado.
Cargadas 313 stopwords en español.
Modelo 'es_core_news_sm' de spaCy cargado.


In [4]:

# --- 1. Configuración de Rutas ---
TRAIN_PATH = os.path.join("..", "data", "processed", "train.csv")
RESULTS_PATH = os.path.join("..", "results", "fase_3_mlp_results.csv")

os.makedirs(os.path.dirname(RESULTS_PATH), exist_ok=True)
warnings.filterwarnings("ignore", category=UserWarning)

# --- 2. Cargar y Preparar Datos ---
print(f"Iniciando Fase 3: MLP + GridSearchCV")
try:
    df_train = pd.read_csv(TRAIN_PATH)
    print(f"Datos de entrenamiento cargados desde '{TRAIN_PATH}'.")
except FileNotFoundError:
    print(f"\n¡Error de Archivo! No se encontró '{TRAIN_PATH}'.")
    exit()

print("Aplicando 'pipeline_d_normalize' (Combinación Ganadora) a los datos...")
# Aplicar la normalización GANADORA
X_train = df_train['text'].apply(pipeline_d_normalize)
y_train = df_train['Polarity']
print(f"Datos normalizados listos. {len(X_train)} documentos.")


# --- 3. Definir Pipeline y Rejilla de Búsqueda ---

# Crear el Pipeline con la combinación GANADORA
text_pipeline = Pipeline([
    ('features', get_augmented_features()), 
    ('classifier', MLPClassifier(random_state=0, max_iter=500, early_stopping=True))
])

# Rejilla Pequeña y Rápida (Total combinaciones: 4)
param_grid = {
    # Probamos si necesita más neuronas para procesar la info extra
    'classifier__hidden_layer_sizes': [(50,), (100,)], 
    
    # Probamos si la regularización anterior (0.01) era muy fuerte
    # 0.0001 es el default de sklearn, suele funcionar bien
    'classifier__alpha': [0.0001, 0.01],
    
    'classifier__activation': ['relu'],
    'classifier__solver': ['adam']
}
print(f"\nIniciando Grid Quirúrgico...")
print(f"Total de entrenamientos: {len(param_grid['classifier__hidden_layer_sizes']) * len(param_grid['classifier__alpha']) * 5} (20 fits)")

Iniciando Fase 3: MLP + GridSearchCV
Datos de entrenamiento cargados desde '..\data\processed\train.csv'.
Aplicando 'pipeline_d_normalize' (Combinación Ganadora) a los datos...
Datos normalizados listos. 24169 documentos.

Iniciando Grid Quirúrgico...
Total de entrenamientos: 20 (20 fits)


In [5]:

# --- 4. Configurar y Ejecutar Validación Cruzada ---

print("\n--- INICIANDO GRIDSEARCH ---")

grid_search = GridSearchCV(
    text_pipeline, 
    param_grid, 
    cv=5, 
    scoring='f1_macro',
    n_jobs=-1,
    verbose=10 # Para ver que avanza
)

try:
    grid_search.fit(X_train, y_train)
except Exception as e:
    print(f"\n¡Error durante el entrenamiento de MLP! {e}")
    exit()

print("\n--- ¡GridSearchCV completado! ---")

# --- 5. Guardar y Reportar Resultados ---

best_score = grid_search.best_score_
best_params = grid_search.best_params_

print(f"Mejor F1-Macro (promedio de CV): {best_score:.4f}")
print(f"Mejores Hiperparámetros encontrados: {best_params}")

# Guardar los resultados
results_df = pd.DataFrame({
    "model": "MLPClassifier",
    "pipeline_base": "D (Negación) + Binary + Lexicons",
    "best_avg_f1_macro": [best_score],
    "best_params": [str(best_params)] # Guardar como string
})

results_df.to_csv(RESULTS_PATH, index=False)
print(f"Resultados de la Fase 3 guardados en '{RESULTS_PATH}'")


--- INICIANDO GRIDSEARCH ---
Fitting 5 folds for each of 4 candidates, totalling 20 fits
--- [lexicon_utils] Cargando recursos lingüísticos ---

--- ¡GridSearchCV completado! ---
Mejor F1-Macro (promedio de CV): 0.4354
Mejores Hiperparámetros encontrados: {'classifier__activation': 'relu', 'classifier__alpha': 0.01, 'classifier__hidden_layer_sizes': (100,), 'classifier__solver': 'adam'}
Resultados de la Fase 3 guardados en '..\results\fase_3_mlp_results.csv'
